In [1]:
import numpy as np
import pandas as pd

In [2]:
alldata = pd.read_excel("E:\\Capstone Project\\Data\\Taxi_area\\drop202004final.xlsx")
data1 = alldata.groupby(['LocationID']).mean()
data1 = data1[['Freq','pop_density','commercial_density','road_density','X','Y']]

from sklearn import preprocessing

scaler = preprocessing.StandardScaler()
scaler.fit(data1[['pop_density','commercial_density','road_density']])
data1[['pop_density','commercial_density','road_density']] = scaler.transform(data1[['pop_density','commercial_density','road_density']])

In [3]:
X = np.array(data1[["X"]])
Y = np.array(data1[["Y"]])
coords = np.hstack([X,Y])

y = np.array(data1[["Freq"]])

pop = np.array(data1[["pop_density"]])
com = np.array(data1[["commercial_density"]])
roa = np.array(data1[["road_density"]])
x = np.hstack([pop,com,roa])

from gtwr.model import GWR

results1 = GWR(coords, y, x,49, kernel='bisquare').fit()

results1.R2

0.886911329084052

In [4]:
coords = results1.coords
coords = pd.DataFrame(coords)

betas = results1.betas
betas = pd.DataFrame(betas)

tvalues = results1.tvalues
tvalues = pd.DataFrame(tvalues)

final_result = pd.concat([betas,tvalues,coords], axis = 1)
final_result.columns=['beta0','POP_beta','COM_beta','ROA_beta','t_beta0','t_POP_beta','t_COM_beta','t_ROA_beta','X','Y']

data2 = pd.merge(data1, final_result, left_on=['X','Y'], right_on=['X','Y'] , how='left')
data2.drop(['Freq','pop_density','commercial_density','road_density'],axis=1,inplace = True)
data2.to_csv(r'E:\\Capstone Project\\Data\\paper\\minor\\drop202004beta.csv',index=True,sep=',')

In [5]:
betas = results1.betas
betas = pd.DataFrame(betas)
betas.describe()

,0,1,2,3
count,257.000000,257.000000,257.000000,257.000000
mean,6.009801,0.753539,1.150452,0.318448
std,7.893417,2.406833,2.514591,2.626072
min,-4.719459,-5.578834,-3.606305,-18.014743
25%,1.814542,-0.195092,-0.045972,-0.290216
50%,2.401662,0.107076,0.301786,0.047996
75%,4.484858,0.532199,1.105740,0.650715
max,33.465373,13.796956,15.675988,9.169528
